<a href="https://colab.research.google.com/github/henryspg/DS-Unit-1-Sprint-2-Statistics/blob/master/LS_DS_212_assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Lambda School Data Science

*Unit 2, Sprint 1, Module 2*

---

# Regression 2

## Assignment

You'll continue to **predict how much it costs to rent an apartment in NYC,** using the dataset from renthop.com.

- [ ] Do train/test split. Use data from April & May 2016 to train. Use data from June 2016 to test.
- [ ] Engineer at least two new features. (See below for explanation & ideas.)
- [ ] Fit a linear regression model with at least two features.
- [ ] Get the model's coefficients and intercept.
- [ ] Get regression metrics RMSE, MAE, and $R^2$, for both the train and test data.
- [ ] What's the best test MAE you can get? Share your score and features used with your cohort on Slack!
- [ ] As always, commit your notebook to your fork of the GitHub repo.


#### [Feature Engineering](https://en.wikipedia.org/wiki/Feature_engineering)

> "Some machine learning projects succeed and some fail. What makes the difference? Easily the most important factor is the features used." — Pedro Domingos, ["A Few Useful Things to Know about Machine Learning"](https://homes.cs.washington.edu/~pedrod/papers/cacm12.pdf)

> "Coming up with features is difficult, time-consuming, requires expert knowledge. 'Applied machine learning' is basically feature engineering." — Andrew Ng, [Machine Learning and AI via Brain simulations](https://forum.stanford.edu/events/2011/2011slides/plenary/2011plenaryNg.pdf) 

> Feature engineering is the process of using domain knowledge of the data to create features that make machine learning algorithms work. 

#### Feature Ideas
- Does the apartment have a description?
- How long is the description?
- How many total perks does each apartment have?
- Are cats _or_ dogs allowed?
- Are cats _and_ dogs allowed?
- Total number of rooms (beds + baths)
- Ratio of beds to baths
- What's the neighborhood, based on address or latitude & longitude?

## Stretch Goals
- [ ] If you want more math, skim [_An Introduction to Statistical Learning_](http://faculty.marshall.usc.edu/gareth-james/ISL/ISLR%20Seventh%20Printing.pdf),  Chapter 3.1, Simple Linear Regression, & Chapter 3.2, Multiple Linear Regression
- [ ] If you want more introduction, watch [Brandon Foltz, Statistics 101: Simple Linear Regression](https://www.youtube.com/watch?v=ZkjP5RJLQF4)
(20 minutes, over 1 million views)
- [ ] Add your own stretch goal(s) !

In [0]:
import pandas as pd
import numpy as np


In [0]:
%%capture
import sys

# If you're on Colab:
if 'google.colab' in sys.modules:
    DATA_PATH = 'https://raw.githubusercontent.com/LambdaSchool/DS-Unit-2-Applied-Modeling/master/data/'
    !pip install category_encoders==2.*

# If you're working locally:
else:
    DATA_PATH = '../data/'
    
# Ignore this Numpy warning when using Plotly Express:
# FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
import warnings
warnings.filterwarnings(action='ignore', category=FutureWarning, module='numpy')

In [0]:
import numpy as np
import pandas as pd

# Read New York City apartment rental listing data
df = pd.read_csv(DATA_PATH+'apartments/renthop-nyc.csv')
assert df.shape == (49352, 34)

# Remove the most extreme 1% prices,
# the most extreme .1% latitudes, &
# the most extreme .1% longitudes
df = df[(df['price'] >= np.percentile(df['price'], 0.5)) & 
        (df['price'] <= np.percentile(df['price'], 99.5)) & 
        (df['latitude'] >= np.percentile(df['latitude'], 0.05)) & 
        (df['latitude'] < np.percentile(df['latitude'], 99.95)) &
        (df['longitude'] >= np.percentile(df['longitude'], 0.05)) & 
        (df['longitude'] <= np.percentile(df['longitude'], 99.95))]

In [414]:
df.tail(3)

,bathrooms,bedrooms,created,description,display_address,latitude,longitude,price,street_address,interest_level,elevator,cats_allowed,hardwood_floors,dogs_allowed,doorman,dishwasher,no_fee,laundry_in_building,fitness_center,pre-war,laundry_in_unit,roof_deck,outdoor_space,dining_room,high_speed_internet,balcony,swimming_pool,new_construction,terrace,exclusive,loft,garden_patio,wheelchair_access,common_outdoor_space
49349,1.0,1,2016-04-16 02:13:40,Large Renovated One Bedroom Apartment with Sta...,West 45th Street,40.7601,-73.9900,2595,341 West 45th Street,low,1,1,0,1,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
49350,1.0,0,2016-04-08 02:13:33,Stylishly sleek studio apartment with unsurpas...,Wall Street,40.7066,-74.0101,3350,37 Wall Street,low,1,1,0,1,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
49351,1.0,2,2016-04-12 02:48:07,Look no further!!! This giant 2 bedroom apart...,Park Terrace East,40.8699,-73.9172,2200,30 Park Terrace East,low,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [415]:
df.describe()

,bathrooms,bedrooms,latitude,longitude,price,elevator,cats_allowed,hardwood_floors,dogs_allowed,doorman,dishwasher,no_fee,laundry_in_building,fitness_center,pre-war,laundry_in_unit,roof_deck,outdoor_space,dining_room,high_speed_internet,balcony,swimming_pool,new_construction,terrace,exclusive,loft,garden_patio,wheelchair_access,common_outdoor_space
count,48817.000000,48817.000000,48817.000000,48817.000000,48817.000000,48817.000000,48817.000000,48817.000000,48817.000000,48817.000000,48817.000000,48817.000000,48817.000000,48817.000000,48817.000000,48817.000000,48817.000000,48817.000000,48817.000000,48817.000000,48817.000000,48817.000000,48817.000000,48817.000000,48817.000000,48817.000000,48817.000000,48817.000000,48817.000000
mean,1.201794,1.537149,40.750760,-73.972760,3579.585247,0.524838,0.478276,0.478276,0.447631,0.424852,0.415081,0.367085,0.052769,0.268452,0.185653,0.175902,0.132761,0.138394,0.102833,0.087203,0.060471,0.055206,0.051908,0.046193,0.043305,0.042711,0.039331,0.027224,0.026241
std,0.470711,1.106087,0.038954,0.028883,1762.430772,0.499388,0.499533,0.499533,0.497255,0.494326,0.492741,0.482015,0.223573,0.443158,0.388830,0.380741,0.339320,0.345317,0.303744,0.282136,0.238359,0.228385,0.221844,0.209905,0.203544,0.202206,0.194382,0.162738,0.159852
min,0.000000,0.000000,40.575700,-74.087300,1375.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,1.000000,1.000000,40.728300,-73.991800,2500.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,1.000000,1.000000,40.751700,-73.978000,3150.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,1.000000,2.000000,40.774000,-73.955000,4095.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,10.000000,8.000000,40.989400,-73.700100,15500.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [0]:
# Check loations

import folium

m = folium.Map(location = [df['latitude'].mean(), df['longitude'].mean()], zoom_start=10,width=1000, height=600 )

def add_marker(row):
  marker = folium.CircleMarker([row['latitude'], row['longitude']], radius=2, color='darkgreen')
  marker.add_to(m)

df.apply(add_marker, axis=1)
m

### Create new columns based on:  
### 1. total number of rooms: bed+bath+dining + balcony  
### 2. dog_allowed + cat_allowed  
### 3. elevator + swimming_pool + fitness_center

In [0]:
df['rooms_main'] = df['bedrooms'] + df['bathrooms']  +  df['dining_room'] + df['terrace']
df['rooms_add'] = df['balcony'] + df['roof_deck']  +  df['loft'] + df['terrace'] + df['garden_patio']
df['pets'] = df['dogs_allowed']*10 + df['cats_allowed']
df['community'] = df['swimming_pool']*100 + df['fitness_center']*10 + df['common_outdoor_space']

df.head(3)

In [0]:
df['rooms_main'].value_counts()

In [0]:


import plotly.express as px
px.scatter(df, x='rooms_main', y='price')

In [0]:
# After combining all 'rooms' we have more variables and max price_line = >10K

px.scatter(df, x='rooms_main', y = 'price', trendline = 'ols')


In [0]:
# Check if there is "exclusive" with   #bedroom=0
# df[(df['exclusive']==1) &  (df['bedrooms']==1)]

# df[(df['exclusive']==0) &  (df['rooms_main']==0)]['price']


In [0]:
# 1st:  Start Train-Test Split with:   #rooms >3  and <=3, and see the error later on

train = df[df['rooms_main'] > 2]
test = df[df['rooms_main'] <= 2]

train.shape,  test.shape

In [0]:
train['price'].mean()

In [0]:
# Arrange target vectors

target = 'price'

y_train = train[target]
y_test = test[target  ]

In [0]:
y_train, y_test

In [0]:
# Base line
# Our target is: PRICE

print("train  :", train['price'].mean(), "\n")
print("test   :", test['price'].mean())

In [0]:
# Mean baseline using 0 feature

guess= y_train.mean()
guess

In [0]:
y_test.mean()

In [0]:
# Train & tets  error

# Train error

from sklearn.metrics import mean_absolute_error
y_pred = [guess] * len(y_train)
mae = mean_absolute_error(y_train, y_pred)
print(f'Train error: {mae:2f} dollar')

In [0]:
# Test error

y_pred = [guess] * len(y_test)
mae = mean_absolute_error(y_test, y_pred)
print(f'Test Error : {mae:.2f}  points')

In [0]:
guess * len(y_test)

In [0]:
mae

In [0]:
# 2nd:  Start Train-Test Split again with:   #rooms >1  and <=1, and see the error later on

train = df[df['rooms_main'] > 1]
test = df[df['rooms_main'] <= 1]

train.shape,  test.shape

In [0]:
train['price'].mean()

In [0]:
# Arrange y target vectors

target = 'price'

y_train = train[target]
y_test = test[target  ]

In [0]:
y_train, y_test

In [0]:
# Base line
# Our target is: PRICE

print("train  :", train['price'].mean(), "\n")
print("test   :", test['price'].mean())

In [0]:
# Mean baseline using 0 feature

guess= y_train.mean()
guess

In [0]:
y_test.mean()

In [0]:
# Train & tets  error

# Train error

from sklearn.metrics import mean_absolute_error
y_pred = [guess] * len(y_train)
mae = mean_absolute_error(y_train, y_pred)
print(f'Train error: {mae:2f} dollar')

In [0]:
# Test error

y_pred = [guess] * len(y_test)
mae = mean_absolute_error(y_test, y_pred)
print(f'Test Error : {mae:.2f}  points')

In [0]:
guess * len(y_test)

In [0]:
mae

After running with  
train = df[df['rooms'] > 3] and test = df[df['rooms'] <= 3] : mae = 1581  
train = df[df['rooms'] > 1] and test = df[df['rooms'] <= 1] : mae = 1401  
  
  Still a high error though !!!!



###Simple regression, 1 feature

In [0]:
import pandas as pd
import plotly.express as px

In [0]:
px.scatter(
    train,
    x='community',
    y='price',
    text='bedrooms',
    title='Apartment price prediction based on community place',
    trendline='ols',  # Ordinary Least Squares
)

###5 steps process for simple regression

In [0]:
# 1. import estimator class from Sklearn

from sklearn.linear_model import LinearRegression

In [0]:
# 2. Instantiate class
model = LinearRegression()
model

In [0]:
# 3. Arrange X-feature matrices

features = ['pets']
X_train = train[features]
X_test = test[features]
print("Linear Regression, dependent on:", features)

In [0]:
# 4. Fit the model & Check train error - how close the model to the train

model.fit(X_train, y_train)

# 4a. Check train error
y_pred = model.predict(X_train)
mae = mean_absolute_error(y_train, y_pred)
print(f'Train Error: {mae:.2f}  points')

In [0]:
# 5. Apply the model to the new data
y_pred = model.predict(X_test)
mae = mean_absolute_error(y_test, y_pred)
print(f'Test Error: {mae:.2f}  points')

##Multiple regression (2 features)  
Make scatterplot of relations between 2 features & target.  
Add feature: rooms_add: patio + balcony + terrace

In [0]:
px.scatter_3d(
    train,
    x='community', 
    y='rooms_add', 
    z='price', 
    text='pets', 
    title='Community places & additional rooms   0: no pets, 01: only cat allowed, 10: only dog allowed, 11: dog & cat allowed'
)

###Use scikit-learn to fit a multiple regression with two features.

In [0]:
# Re-arrange X features matrices

features = ['community', 
            'rooms_add']
print(f'Linear Regression, dependent on: {features}')

X_train = train[features]
X_test = test[features]

In [0]:
# We need to fit the model
model_full = LinearRegression()
model_full.fit(X_train, y_train)

In [0]:
# Calculate training error
y_pred = model_full.predict(X_train)
mean_absolute_error(y_train, y_pred)

In [0]:
# apply the model to new data
y_pred = model_full.predict(X_test)
mean_absolute_error(y_test, y_pred)

###Compare the error to the prior model  
###Plot the plane of best fit

In [0]:
import itertools
import numpy as np
import plotly.express as px
import plotly.graph_objs as go
from sklearn.linear_model import LinearRegression

def regression_3d(df, x, y, z, num=100, **kwargs):
    """
    Visualize linear regression in 3D: 2 features + 1 target
    
    df : Pandas DataFrame
    x : string, feature 1 column in df
    y : string, feature 2 column in df
    z : string, target column in df
    num : integer, number of quantiles for each feature
    """
    
    # Plot data
    fig = px.scatter_3d(df, x, y, z, **kwargs)
    
    # Fit Linear Regression
    features = [x, y]
    target = z
    model = LinearRegression()
    model.fit(df[features], df[target])    
    
    # Define grid of coordinates in the feature space
    xmin, xmax = df[x].min(), df[x].max()
    ymin, ymax = df[y].min(), df[y].max()
    xcoords = np.linspace(xmin, xmax, num)
    ycoords = np.linspace(ymin, ymax, num)
    coords = list(itertools.product(xcoords, ycoords))
    
    # Make predictions for the grid
    predictions = model.predict(coords)
    Z = predictions.reshape(num, num).T
    
    # Plot predictions as a 3D surface (plane)
    fig.add_trace(go.Surface(x=xcoords, y=ycoords, z=Z))
    
    return fig

In [0]:
regression_3d(
    train,
    x='community', 
    y='rooms_add', 
    z='price', 
    text='pets', 
    title='Community places & additional rooms.  0: no pets, 01: only cat allowed, 10: only dog allowed, 11: dog & cat allowed'
)

##Get and interpret coefficients

In [0]:
model_full.intercept_, model_full.coef_

In [0]:
beta0 = model_full.intercept_
beta1, beta2 = model_full.coef_
print(f'y = {beta0} + {beta1}x1 + {beta2}x2')

In [0]:
# better to read

print('Intercept', model_full.intercept_)
coefficients = pd.Series(model_full.coef_, features)
print(coefficients.to_string())

In [0]:
X_train.describe()

###See further

In [0]:
model = model_full

In [0]:
# What if  community=0  & rooms_add=0??

model.predict([[0, 0]])

In [0]:
# What if  community=2  & rooms_add=3 ??

model.predict([[2, 3]])

In [0]:
# The difference between 2,3  & 0,0:   >>  BIG!!!!!

model.predict([[2, 3]]) - model.predict([[0, 0]])

In [0]:
# What if  community=2  & rooms_add=3 ??

model.predict([[3, 2]])

In [0]:
# The difference between 2,3  & 3,2:   >>  BIG!!!!!

model.predict([[2, 3]]) - model.predict([[3, 2]])

#Conclusion:
#Using  Additional rooms and Community utilities  are not a good reason for apartment price prediction. But this is a good learning process !!!